# Imports, paths, and parameters

In [ ]:
# Import utils
import numpy as np
import pandas as pd
import copy
import time
import datetime as dt
import pickle
import json
from pathlib import Path
import joblib
from joblib import dump, load, Parallel, delayed
import os
import itertools
import contextlib
from tqdm import tqdm

# sklearn
from sklearn.preprocessing import MinMaxScaler

# Import Weights Model
import WeightsModel2
from WeightsModel2 import RandomForestWeightsModel
from WeightsModel2 import PreProcessing

# Import (Rolling Horizon) Weighted SAA models
from WeightedSAA2 import WeightedSAA
from WeightedSAA2 import RobustWeightedSAA
from WeightedSAA2 import RollingHorizonOptimization

In [ ]:
# Set folder names as global variables
os.chdir('/home/fesc/MM/')
global PATH_DATA, PATH_PARAMS, PATH_KERNELS, PATH_SAMPLES, PATH_RESULTS

PATH_DATA = '/home/fesc/MM/Data'
PATH_PARAMS  = '/home/fesc/MM/Data/Params'
PATH_WEIGHTSMODEL = '/home/fesc/MM/Data/WeightsModel'
PATH_SAMPLES = '/home/fesc/MM/Data/Samples'
PATH_RESULTS = '/home/fesc/MM/Data/Results'

In [ ]:
# Time period and SKU ranges
T = 13                  # Planning horizon T
ts = range(1,13+1)      # Periods t=1,...,T of the planning horizon
taus = range(0,4+1)     # Look-aheads tau=0,...,4 to use
SKUs = range(1,460+1)   # Products (SKUs) k=1,...,M

# Train/test split (first timePeriods of testing horizon)
test_start = 114

# Global Models

The two global models (using 'Global Training and Sampling') are **Rolling Horizon Global Weighted SAA (GwSAA)**, which is our model, and **Rolling Horizon Global Robust Weighted SAA (GwSAA-R)**, which is the analogous model with robust extension.

Given product $k$, period $t$, and look-ahead $\tau$, both models apply Weighted SAA over the 'global' distribution $\{\{w_{j,t,\tau}^{\,i}(x_{j,t}^{\,i}),(d_{j,t}^{\,i},...,d_{j,t+\tau}^{\,i})\}_{i=1}^{N_{j,t,\tau}}\}_{j=1}^{M}$, with weight functions $w_{j,t,\tau}(\,\cdot\,)$ trained (once for all products) on data $S_{t,\tau}^{\,\text{Global}}=\{\{(x_{j,t}^{\,i},d_{j,t}^{\,i},...,d_{j,t+\tau}^{\,i})\}_{i=1}^{N_{j,t,\tau}}\}_{j=1}^{M}$.

## Data Preprocessing

We first load and pre-process the data. This includes reshaping demand time series into $(\tau+1)$-periods rolling look-ahead horizon sequences. In particular for the global model, we furthermore apply scaling to relevant variables using sklearn's 'MinMaxScaler' fitted on the training horizon.

- **ID_Data** (pd.DataFrame) stores identifiers (in particular the product (SKU) identifier and the timePeriod (sale_yearweek) identifier)
- **X_Data** (pd.DataFrame) is the 'feature matrix', i.e., each row is a feature vector $x_{j,n}$ where n is the number of training observations (rows) in the data
- **Y_Data** (pd.DataFrame) is the demand data $d_{j,n}$ (a times series per product)
- **X_Data_Columns** (pd.DataFrame) provides 'selectors' for local vs. global feature sets

In [ ]:
# Initialize preprocessing module
pp = PreProcessing()

In [ ]:
# Read data
ID_Data = pd.read_csv(PATH_DATA+'/ID_Data.csv')
X_Data = pd.read_csv(PATH_DATA+'/X_Data.csv')
X_Data_Columns = pd.read_csv(PATH_DATA+'/X_Data_Columns.csv')
Y_Data = pd.read_csv(PATH_DATA+'/Y_Data.csv')

In [ ]:
# Select features
X_Data_Columns = X_Data_Columns.loc[X_Data_Columns.Global == 'YES']
X_Data = X_Data[X_Data_Columns.Feature.values]

In [ ]:
# Ensure data is sorted by SKU and sale_yearweek for preprocessing
data = pd.concat([ID_Data, X_Data, Y_Data], axis=1).sort_values(by=['SKU', 'sale_yearweek']).reset_index(drop=True)

ID_Data = data[ID_Data.columns]
X_Data = data[X_Data.columns]
Y_Data = data[Y_Data.columns]

### Scaling - Features

Selected relevant features (as defined by the meta data table 'X_Data_Columns') data are scaled (by product) for training using sklean's min-max scaling method (scaler is fitted once by product on the initial training horizon before model application over periods $t=1,...,T$).

In [ ]:
# Select training data
ID_Data_train = ID_Data.loc[ID_Data.sale_yearweek < test_start]
X_Data_train = X_Data.loc[ID_Data.sale_yearweek < test_start]

# Prepare
vars_to_scale_names = X_Data_Columns.loc[X_Data_Columns.Scale == 'YES', 'Feature'].values
vars_to_scale_with_names = X_Data_Columns.loc[X_Data_Columns.Scale == 'YES', 'ScaleWith'].values

vars_to_scale = np.array(X_Data[vars_to_scale_names])
vars_to_scale_with = np.array(X_Data_train[vars_to_scale_with_names])

vars_to_scale_groups = np.array(ID_Data.SKU)
vars_to_scale_with_groups = np.array(ID_Data_train.SKU)

# Fit and transform
scaler = MinMaxScaler()
vars_scaled, scaler_fitted = pp.scale_variables(vars_to_scale, vars_to_scale_with, vars_to_scale_groups, vars_to_scale_with_groups, scaler)

# Reshape to original data
vars_scaled = pd.concat([pd.DataFrame(vars_scaled[i], columns=vars_to_scale_names) for i in vars_scaled]).reset_index(drop=True)
X_Data_z = copy.deepcopy(X_Data)
for col in vars_scaled.columns:
    X_Data_z[col] = vars_scaled[col]

### Scaling - Demands

Demands for training are scaled (by produc) using sklean's min-max scaling method (scaler is fitted once by product on the initial training horizon before model application over periods $t=1,...,T$).

In [ ]:
# Select training data
ID_Data_train = ID_Data.loc[ID_Data.sale_yearweek < test_start]
Y_Data_train = Y_Data.loc[ID_Data.sale_yearweek < test_start]

# Prepare
vars_to_scale = np.array(Y_Data)
vars_to_scale_with = np.array(Y_Data_train)

vars_to_scale_groups = np.array(ID_Data.SKU)
vars_to_scale_with_groups = np.array(ID_Data_train.SKU)

# Fit and transform
scaler = MinMaxScaler()
vars_scaled, scaler_fitted = pp.scale_variables(vars_to_scale, vars_to_scale_with, vars_to_scale_groups, vars_to_scale_with_groups, scaler)

# Reshape to original data
Y_Data_z = pd.concat([pd.DataFrame(vars_scaled[i], columns=['Y']) for i in vars_scaled]).reset_index(drop=True)

### Reshape to multi-period demand

We now reshape the time series of demands per product to consecutive $(\tau+1)$-periods demand vectors.

In [ ]:
# Create multi-period demand vectors
data = pd.concat([ID_Data, Y_Data_z], axis=1)
Y = {}
for tau in taus:
    Y['Y'+str(tau)] = data.groupby(['SKU']).shift(-tau)['Y']
    
Y_Data_z = pd.DataFrame(Y)

data = pd.concat([ID_Data, Y_Data], axis=1)
Y = {}
for tau in taus:
    Y['Y'+str(tau)] = data.groupby(['SKU']).shift(-tau)['Y']
    
Y_Data = pd.DataFrame(Y)

## Weights model

The weights model - and thus the data used, weight functions, and weights per sample - are the same for the two global models **GwSAA** and **GwSAA-R**. First, we tune the hyper parameters of the random forest weights model for each given look-ahead $\tau$ (as for each look-ahead $\tau$ we have a different response for the multi-output random forest regressor). Second, we fit all weight functions (for each look-ahead $\tau=0,...,4$ and over periods $t=1,...,T$) and generate all weights (for each look-ahead $\tau=0,...,4$, over periods $t=1,...,T$, and for each product (SKU) $k=1,...,M$).

### Tune weights model

To tune the hyper parameters of the global random forest weights model, we use 3-fold rolling timeseries cross-validation on the training data and perform random search with 100 iterations over the specified hyper parameter search grid.

In [ ]:
# Set names
weightsmodel_cv_name = 'cv_rfwm_global'

In [ ]:
# Set parameters to tune random forest weights kernels
model_params = {
    'oob_score': True,
    'random_state': 12345,
    'n_jobs': 4,
    'verbose': 0
}

hyper_params_grid = {
    'n_estimators': [1000],
    'max_depth': [None],
    'min_samples_split': [x for x in range(20, 1000, 20)],  
    'min_samples_leaf': [x for x in range(10, 1000, 10)],  
    'max_features': [x for x in range(8, 256, 8)],   
    'max_leaf_nodes': [None],
    'min_impurity_decrease': [0.0],
    'bootstrap': [True],
    'max_samples': [0.75, 0.80, 0.85, 0.90, 0.95, 1.00]
}    


tuning_params = {     
    'random_search': True,
    'n_iter': 100,
    'scoring': {'MSE': 'neg_mean_squared_error'},
    'return_train_score': True,
    'refit': 'MSE',
    'random_state': 12345,
    'n_jobs': 8,
    'verbose': 2
}    

In [ ]:
# Tune random forest weights models for tau=0,...,4
for tau in taus:
        
    # Select training data
    train = (ID_Data.sale_yearweek < test_start)
    rolling_horizon = [l for l in range(0,tau+1)]
        
    # Select training data
    ID_Data_train = ID_Data.loc[train]
    X_Data_z_train = X_Data_z.loc[train]
    Y_Data_z_train = Y_Data_z.loc[train].iloc[:,rolling_horizon]

    # Reshape to match (tau+1)-periods rolling horizon
    timePeriods = ID_Data_train.sale_yearweek
    maxTimePeriod = test_start-1
    
    id_train = pp.reshape_data(ID_Data_train, timePeriods, maxTimePeriod, tau)
    X_train_z = pp.reshape_data(X_Data_z_train, timePeriods, maxTimePeriod, tau)
    y_train_z = pp.reshape_data(Y_Data_z_train, timePeriods, maxTimePeriod, tau)
    
    # Tansfrom data to arrays
    X_train_z = np.array(X_train_z)
    y_train_z = np.array(y_train_z).flatten() if np.array(y_train_z).shape[1] == 1 else np.array(y_train_z)    

    # Initialize
    weightsmodel = RandomForestWeightsModel()

    # CV folds
    cv_folds = pp.split_timeseries_cv(n_splits=3, timePeriods=id_train.sale_yearweek)
    
    # CV search
    cv_results = weightsmodel.tune(X=X_train_z, y=y_train_z, cv_folds=cv_folds, model_params=model_params, 
                                   tuning_params=tuning_params, hyper_params_grid=hyper_params_grid)
    
    # Save
    weightsmodel.save_cv_result(path=PATH_WEIGHTSMODEL+'/'+weightsmodel_cv_name+'_tau'+str(tau)+'.joblib')

### Fit weights model and generate weights

We now fit the global random forest weights model (i.e., the weight functions) for each $\tau=0,...,4$ and period $t=1,...,T$. This is done across all products at once (global training). Then, for each $\tau=0,...,4$ and period $t=1,...,T$, we generate for each product (SKU) $k=1,...,M$ the weights given the test feature $x_{k,t}$. This is done *jointly* across products for computational efficiency - the weights for each product are extracted afterwards.

In [ ]:
# Set names
weightsmodel_cv_name = 'cv_rfwm_global'
weightsmodel_name = 'rfwm_global'

In [ ]:
# Initialize
samples = {}
weightfunctions = {}
weights = {}
exec_time_sec = {}
cpu_time_sec = {}
        
# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Initialize
    samples[tau] = {}
    weightfunctions[tau] = {}
    weights[tau] = {}
    exec_time_sec[tau] = {}
    cpu_time_sec[tau] = {}

    # For each period t=1,...,T
    for t in ts:
        
        # Adjust look-ahead tau to account for end of horizon
        tau_t = min(tau,T-t)

        # Status
        print('#### Look-ahead tau='+str(tau)+' (adjusted to tau\'='+str(tau_t)+'), period t='+str(t)+' ...')
        start_time = dt.datetime.now().replace(microsecond=0)
        exec_time_sec[tau][t] = {}
        cpu_time_sec[tau][t] = {}
        
        # Select training and test data
        train = (ID_Data.sale_yearweek < test_start+t-1)
        test = (ID_Data.sale_yearweek == test_start+t-1)
        rolling_horizon = [l for l in range(0,tau_t+1)]
        
        ID_Data_train, ID_Data_test = ID_Data.loc[train], ID_Data.loc[test]
        X_Data_z_train, X_Data_z_test = X_Data_z.loc[train], X_Data_z.loc[test]
        Y_Data_z_train, Y_Data_z_test = Y_Data_z.loc[train].iloc[:,rolling_horizon], Y_Data_z.loc[test].iloc[:,rolling_horizon]
        Y_Data_train, Y_Data_test = Y_Data.loc[train].iloc[:,rolling_horizon], Y_Data.loc[test].iloc[:,rolling_horizon]

        # Reshape to match (tau+1)-periods rolling horizon
        timePeriods = ID_Data_train.sale_yearweek
        maxTimePeriod = test_start-1+t-1
    
        id_train = pp.reshape_data(ID_Data_train, timePeriods, maxTimePeriod, tau_t)
        X_train_z = pp.reshape_data(X_Data_z_train, timePeriods, maxTimePeriod, tau_t)
        y_train_z = pp.reshape_data(Y_Data_z_train, timePeriods, maxTimePeriod, tau_t)
        y_train = pp.reshape_data(Y_Data_train, timePeriods, maxTimePeriod, tau_t)
        
        # Tansfrom data to arrays
        X_train_z, X_test_z = np.array(X_train_z), np.array(X_Data_z_test)
        y_train_z, y_test_z = np.array(y_train_z), np.array(Y_Data_z_test)
        y_train, y_test = np.array(y_train), np.array(Y_Data_test) 
        
        if tau_t == 0:
            
            y_train_z, y_test_z = y_train_z.flatten(), y_test_z.flatten()
            y_train, y_test = y_train.flatten(), y_test.flatten() 

        # Check if fit already exists, due to the adjusted look-ahead tau_t:
        if t in weightfunctions[tau_t].keys() if tau_t in weightfunctions.keys() else False:
            
            # Set weightsmodel to fitted weightsmodel already existing
            weightsmodel = weightfunctions[tau_t][t]
            weightfunctions[tau][t] = weightfunctions[tau_t][t]
            exec_time_sec[tau][t]['fit'] = exec_time_sec[tau_t][t]['fit']
            cpu_time_sec[tau][t]['fit'] = cpu_time_sec[tau_t][t]['fit']
            
        else: 
                    
            # Timer start
            st_exec = time.time()
            st_cpu = time.process_time() 
        
            # Initialize weights model
            weightsmodel = RandomForestWeightsModel()

            # Load cv results
            weightsmodel.load_cv_result(path=PATH_WEIGHTSMODEL+'/'+weightsmodel_cv_name+'_tau'+str(tau_t)+'.joblib')

            # Fit random forest weights model
            weightfunctions[tau][t] = weightsmodel.fit(X=X_train_z, y=y_train_z, model_params={'n_jobs': 32, 'verbose': 0})

            # Timer end
            exec_time_sec[tau][t]['fit'] = time.time()-st_exec
            cpu_time_sec[tau][t]['fit'] = time.process_time()-st_cpu

        # Check if weights already exists due to the adjusted look-ahead tau_t:
        if t in weights[tau_t].keys() if tau_t in weights.keys() else False:
            
            # Set weights to weights already existing
            weights[tau][t] = weights[tau_t][t]
            exec_time_sec[tau][t]['weights'] = exec_time_sec[tau_t][t]['weights']
            cpu_time_sec[tau][t]['weights'] = cpu_time_sec[tau_t][t]['weights']
            
        else: 

            # Timer start
            st_exec = time.time()
            st_cpu = time.process_time()  
            
                    
            """
            Note: Each row in X_test belongs to a different product, i.e., we create each n weights for each of the m products
            in X_test at once for the current period and look-ahead.

            """
        
            # Initialize
            weights[tau][t] = {}
            
            # Get weights
            w = weightsmodel.apply(X=X_train_z, x=X_test_z, model_params={'n_jobs': 32, 'verbose': 0})    

            # Store weights for each X_test_z[m,] separately (each corresponding to a test product (SKU))
            for SKU in SKUs:
                
                weights[tau][t][SKU] = w[ID_Data_test.SKU==SKU,].flatten()
                
            # Timer end
            exec_time_sec[tau][t]['weights'] = time.time()-st_exec
            cpu_time_sec[tau][t]['weights'] = time.process_time()-st_cpu
            
            
        # Check if samples already exists due to the adjusted look-ahead tau_t:
        if t in samples[tau_t].keys() if tau_t in samples.keys() else False:
            
            # Set samples to samples already existing
            samples[tau][t] = samples[tau_t][t]
            
        else: 
            
            # Initialize
            samples[tau][t] = {}
            
            # Store unscaled samples for each test product separately
            for SKU in SKUs:
                
                samples[tau][t][SKU] = {'y_train': y_train,
                                        'y_test': y_test[ID_Data_test.SKU==SKU], 
                                        'id_train': id_train,
                                        'id_test': ID_Data_test.loc[ID_Data_test.SKU==SKU]}

        # Status
        print('...done in', dt.datetime.now().replace(microsecond=0) - start_time)    

# Save results
joblib.dump(samples, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples.joblib')    
joblib.dump(weightfunctions, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weightfunctions.joblib')    
joblib.dump(weights, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights.joblib')    
joblib.dump(exec_time_sec, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weightsmodel_exec_time_sec.joblib')  
joblib.dump(cpu_time_sec, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weightsmodel_cpu_time_sec.joblib')  

## Optimization

Description ...

### Data preparation

We first define a function that prepares the data needed for an experiment (depending on the model/approach). 

If no sampling strategy is provided via the optional argument 'sampling', no weights are retrieved, else 'global' or 'local' weights are retrieved and historical demands are prepared for 'global' or 'local' sampling, respectively. 
    
If the optional argument 'e' is provided, the function additionally outputs 'epsilon' which is the uncertainty set threshold for robust optimization.

In [ ]:
## Prepare the data to a run an experiment over the full planning horizon
def prep_data(SKU, tau, T, sale_yearweek, path_data, path_samples, **kwargs):

    """

    This function prepares the data needed for (weighted, robust) optimization. If no sampling strategy is
    provided via the optional argument 'sampling', no weights are retrieved, else 'global' or 'local' weights
    are retrieved and historical demands are prepared for 'global' or 'local' sampling, respectively. If the
    optional argument 'e' is provided, the function additionally outputs 'epsilon' which is the uncertainty
    set threshold for robust optimization.

    Arguments:

        SKU: product (SKU) identifier
        tau: length of rolling look-ahead horizon
        T: Length T of the test horizon
        sale_yearweek: Last sale_yearweek of training data
        path_data: path of data
        path_samples: path of samples

    Optional arguments: 

        sampling: Sampling strategy (either 'global', 'local'), with
            - 'global': uses weights generated with global training
            - 'local': uses weights generated with local training
        e: Robust uncertainty set threshold multiplier, with
            - int: uses e as multiplier for product's in sample standard deviation as the uncertainty set threshold 

    Output:

        y: demand data - np.array of shape (n_samples, n_periods)
        ids_train: list of selector series (True/False of length n_samples) - list with lengths of the test horizon
        ids_test: list of selector series (True/False of length n_samples) - list with lengths of the test horizon

        weights (optional): list of weights (flat np.array of length ids_train of t'th test period) - list 
        with length of test horizon
        epsilons (optional): list of epsilons - list with length of the test horizon

    """

    # Demand samples
    robj = pyreadr.read_r(path_data+'/Y_Data_mv_NEW.RData')
    y_samples = np.array(robj['Y_Data_mv'])

    # IDs of local demand samples
    robj = pyreadr.read_r(path_data+'/ID_Data_NEW.RData')
    ID_samples = robj['ID_Data']

    # IDs of local demand samples
    robj = pyreadr.read_r(path_samples+'/SKU'+str(SKU)+'/Static/TmpFiles'+
                          str(tau)+'/ID_samples_k.RDS')
    ID_samples_SKU = robj[None]

    # If sampling strategy is provided
    if 'sampling' in kwargs:

        # Weights
        with open(path_samples+'/SKU'+str(SKU)+'/Static/Weights'+
                  str(tau)+'/weights_'+kwargs['sampling']+'_ij.p', 'rb') as f:
            weighty_ij = pickle.load(f)
        del f

        # Demand samples for global sampling
        if kwargs['sampling'] == 'global':
            y = y_samples

        # Demand samples for local sampling
        if kwargs['sampling'] == 'local':
            y = y_samples[ID_samples.SKU_API == ID_samples_SKU.SKU_API[0]]

    # Default: local demand samples
    else:
        y = y_samples[ID_samples.SKU_API == ID_samples_SKU.SKU_API[0]]


    # Reshape data for each t=1...T (i.e., each period of the test horizon)
    ids_train = []
    ids_test = []

    weights = [] if 'sampling' in kwargs else None
    epsilons = [] if 'e' in kwargs else None

    # Iterate over t
    for t in range(T):

        # If sampling strategy is provided
        if 'sampling' in kwargs:

            # IDs of demand samples for global sampling
            if kwargs['sampling'] == 'global':
                ids_train = ids_train + [ID_samples.sale_yearweek < sale_yearweek+t]

            # IDs of demand samples for local sampling
            if kwargs['sampling'] == 'local':
                ids_train = ids_train + [(ID_samples.SKU_API == ID_samples_SKU.SKU_API[0]) &
                                         (ID_samples.sale_yearweek < sale_yearweek+t)]                   

            # Weights for global/local
            weights = weights + [weighty_ij[t+1]]

        # Default: IDs of demand samples for local sampling
        else:
            ids_train = ids_train + [(ID_samples.SKU_API == ID_samples_SKU.SKU_API[0]) &
                                         (ID_samples.sale_yearweek < sale_yearweek+t)]



        # IDs of demand samples for testing 
        ids_test = ids_test + [(ID_samples.SKU_API == ID_samples_SKU.SKU_API[0]) &
                                         (ID_samples.sale_yearweek == sale_yearweek+t)]


        # If e is provided, calculate robust optimization parameter epsilon
        if 'e' in kwargs:
            epsilons = epsilons + [kwargs['e']*np.std(y_samples[(ID_samples.SKU_API == ID_samples_SKU.SKU_API[0]) &
                                                                (ID_samples.sale_yearweek < sale_yearweek+t),0])]


    # Return
    return y, ids_train, ids_test, weights, epsilons

### Experiment wrapper

We now define a 'wrapper' function that iterates the experiment for a given SKU over differen cost parameter settins and lengths of the rolling look-ahead horizon tau.

If the parameter sampling is provided (either 'global' or 'local), the function uses the specified sampling strategy. Else, SAA is performed. If the multiplier 'e' for the uncertainty set threshold epsilon is provided, the function performs the robust extension.
 
Where: epsilon[t] = e *  in-sample standard deviation of the current product (SKU).

The function prepares and calls the experiment over t=1...T for each cost paramater setting and look-ahead horizon tau and then summarises the results including performance and performance meta inormation. It also saves the results in CSV format to the specified path and the function can also be used in parallel processing environments.

In [ ]:
def run_experiments(SKU, **kwargs):
    
    """
    
    Description ...
    
    
    Arguments:
    
        SKU: product (SKU) identifier
        sale_yearweek: Last sale_yearweek of training data
        T: Length T of the test horizon
        tau: List of lengths of rolling look-ahead horizons
        cost_params: dictionary/dictionary of dictionaries of cost parameters {'CR', 'K', 'u', 'h', 'b'}
        gurobi_params: dictionary of gurobi meta params {'LogToConsole', 'Threads', 'NonConvex' 
                                                         'PSDTol', 'MIPGap', 'NumericFocus',
                                                         'obj_improvement', obj_timeout_sec'}
        path: directory where results should be saved
        model_name: model name for the file to save results
        
    Optional arguments:
    
        sampling: sampling strategy (either 'global' or 'local'); performs SAA if not provided
        e: robust uncertainty set threshold multiplier; performs no robust extension if not provided
    

    """
  
    st_exec = time.time()
    st_cpu = time.process_time()
    
    # Print progress
    if kwargs['print_progress']: 
        print('SKU:', SKU)
    
    # Initialize
    rhopt = RollingHorizonOptimization()
    results = pd.DataFrame()
   
    # For each cost param setting
    for cost_params in kwargs['cost_params'].values():
        
        # Print progress
        if kwargs['print_progress']: 
            print('... cost param setting:', cost_params)
    
        # For each rolling look-ahead horizon
        for tau in kwargs['tau']:
            
            # Print progress
            if kwargs['print_progress']: 
                print('...... look-ahead horizon:', tau)
    
            ## Weighted SAA
            if 'sampling' in kwargs:
    
                ## Weighted Robust SAA  
                if 'e' in kwargs:

                    # Prepare data
                    data = prep_data(SKU, tau, kwargs['T'], kwargs['sale_yearweek'], PATH_DATA, PATH_SAMPLES, sampling=kwargs['sampling'], e=kwargs['e'])
                    y, ids_train, ids_test, weights, epsilons = data
                    
                    # Create empty model
                    wsaamodel = RobustWeightedSAA(**kwargs['gurobi_params'])

                    # Run rolling horizon model over t=1...T
                    result = rhopt.run(y, ids_train, ids_test, tau, wsaamodel, weights=weights, epsilons=epsilons, **cost_params)

                ## Weighted SAA
                else: 
                    
                    # Prepare data
                    data = prep_data(SKU, tau, kwargs['T'], kwargs['sale_yearweek'], PATH_DATA, PATH_SAMPLES, sampling=kwargs['sampling'])
                    y, ids_train, ids_test, weights, _ = data
                    
                    # Create empty model
                    wsaamodel = WeightedSAA(**kwargs['gurobi_params'])

                    # Run rolling horizon model over t=1...T
                    result = rhopt.run(y, ids_train, ids_test, tau, wsaamodel, weights=weights, **cost_params)


            ## SAA
            else:
                
                # Prepare data
                data = prep_data(SKU, tau, kwargs['T'], kwargs['sale_yearweek'], PATH_DATA, PATH_SAMPLES)
                y, ids_train, ids_test, _, _ = data

                # Create empty model
                wsaamodel = WeightedSAA(**kwargs['gurobi_params'])

                # Run rolling horizon model over t=1...T
                result = rhopt.run(y, ids_train, ids_test, tau, wsaamodel, **cost_params)

            
            ## ToDo: ExPost
            
            # Store result
            meta = pd.DataFrame({

                'SKU': np.repeat(SKU,kwargs['T']),
                'n_periods': np.repeat(kwargs['T'],kwargs['T']),
                'tau': np.repeat(tau,kwargs['T']),
                'CR': np.repeat(cost_params['CR'],kwargs['T']),
                'LogToConsole': np.repeat(kwargs['gurobi_params']['LogToConsole'],kwargs['T']),
                'Threads': np.repeat(kwargs['gurobi_params']['Threads'],kwargs['T']),
                'NonConvex': np.repeat(kwargs['gurobi_params']['NonConvex'],kwargs['T']),
                'PSDTol': np.repeat(kwargs['gurobi_params']['PSDTol'],kwargs['T']),
                'MIPGap': np.repeat(kwargs['gurobi_params']['MIPGap'],kwargs['T']),
                'NumericFocus': np.repeat(kwargs['gurobi_params']['NumericFocus'],kwargs['T']),
                'obj_improvement': np.repeat(kwargs['gurobi_params']['obj_improvement'],kwargs['T']),
                'obj_timeout_sec': np.repeat(kwargs['gurobi_params']['obj_timeout_sec'],kwargs['T']),
                'e': np.repeat(kwargs['e'],kwargs['T']) if 'e' in kwargs else np.repeat(0,kwargs['T']),
                'epsilon': [epsilon for epsilon in epsilons] if 'e' in kwargs else np.repeat(0,kwargs['T'])
            })

            result = pd.concat([meta, result], axis=1)

            # Store
            if not results.empty:
                results = results.append(result)   
            else:
                results = pd.DataFrame(result) 

    # Save result
    save_log = results.to_csv(
        path_or_buf=kwargs['path']+'/'+kwargs['model_name']+'_SKU'+str(SKU)+(('_e'+str(kwargs['e'])) if 'e' in kwargs else '')+'.csv', 
        sep=',', index=False
    )
    
    
    # Time
    exec_time_sec = time.time() - st_exec
    cpu_time_sec = time.process_time() - st_cpu
    
    # Print progress
    if kwargs['print_progress']: 
        print('>>>> Done:',str(np.around(exec_time_sec/60,1)), 'minutes')

    
    # Returns results 
    if (kwargs['return_results'] if 'return_results' in kwargs else False):
        return results
    
    # Returns a log
    else:
        return  {'SKU': SKU, 'exec_time_sec': exec_time_sec, 'cpu_time_sec': cpu_time_sec}

### Context Manager

This is a context manager for parellel execution with the purpose of reporting progress. 

Credits: https://stackoverflow.com/questions/24983493/tracking-progress-of-joblib-parallel-execution

In [ ]:
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

### Rolling Horizon Global Weighted SAA (GwSAA)

Description ...

The code below runs an experiment for all given products (SKUs) $k=1,...,M$ over a test planning horizon $t=1,...,T$ with $T=13$ for three different cost parameter settings $\{K, u, h, b\}$ that vary the critical ratio ($CR=\frac{b}{b+h}$) of holding and backlogging yielding
- $CR=0.50$: $\{K=100, u=0.5, h=1, b=1\}$
- $CR=0.75$: $\{K=100, u=0.5, h=1, b=3\}$
- $CR=0.90$: $\{K=100, u=0.5, h=1, b=9\}$






In [ ]:
def run_experiments(wsaamodel, samples, weights=None, epsilons=None, cost_params=None, print_progress=False, 
                    path_to_save=None, name_to_save=None, return_results=True, **kwargs):
    
    """
    ...
    
    """

    # Timer
    st_exec = time.time()
    st_cpu = time.process_time()
    
    # Status
    if print_progress and 'SKU' in kwargs: 
        print('SKU:', kwargs['SKU'])
    
    # Initialize
    ropt = RollingHorizonOptimization()
    results = pd.DataFrame()
   
    # If cost params are provided
    if not cost_params is None:
        
        # If provided, has to be a list of dict(s)
        if type(cost_params)==list:

            # For each cost param setting (if provied) 
            for cost_params_ in cost_params:

                # Print progress
                if print_progress: print('... cost param setting:', cost_params_)

                # Apply (Weighted) SAA  model
                wsaamodel.set_params(**{**kwargs, **cost_params_})
                result = ropt.run2(wsaamodel, samples, weights, epsilons)

                ## Store results
                meta = pd.DataFrame({'CR': cost_params_['CR'], **kwargs}, index=list(range(len(samples))))
                results = pd.concat([results, pd.concat([meta, result], axis=1)], axis=0)
                
        else:
            
            # Raise error if cost params is not None but also no list
            raise ValueError('If provided, cost_params has to be a list of at least one dict with keys {K, u, h, b}')
    
    # If no cost params provided, run with model's current params (default or as initialized)
    else:

        # Apply (Weighted) SAA  model
        wsaamodel.set_params(**kwargs)
        result = ropt.run2(wsaamodel, samples, weights, epsilons)

        ## Store results
        meta = pd.DataFrame(kwargs, index=list(range(len(samples))))
        results = pd.concat([results, pd.concat([meta, result], axis=1)], axis=0)


    
    
    # Save result as csv file
    if not path_to_save is None and not name_to_save is None:
        results.to_csv(path_or_buf=path_to_save+'/'+name_to_save+'_SKU'+str(SKU)+'.csv', sep=',', index=False)

    
    # Timer
    exec_time_sec = time.time() - st_exec
    cpu_time_sec = time.process_time() - st_cpu
    
    # Status
    if print_progress: 
        print('>>>> Done:',str(np.around(exec_time_sec/60,1)), 'minutes')

    # Return  
    output = results if return_results else {'SKU': SKU, 'exec_time_sec': exec_time_sec, 'cpu_time_sec': cpu_time_sec}
    return output

In [ ]:
# Define experiment paramaters
experiment_params = {
            
    # Cost param settings
    'cost_params': [

        {'CR': 0.50, 'K': 100, 'u': 0.5, 'h': 1, 'b': 1},
        {'CR': 0.75, 'K': 100, 'u': 0.5, 'h': 1, 'b': 3},
        {'CR': 0.90, 'K': 100, 'u': 0.5, 'h': 1, 'b': 9}

    ],

    # Gurobi meta params
    'LogToConsole': 0, 
    'Threads': 1, 
    'NonConvex': 2, 
    'PSDTol': 1e-3, # 0.1%
    'MIPGap': 1e-3, # 0.1%
    'NumericFocus': 0, 
    'obj_improvement': 1e-3, # 0.1%
    'obj_timeout_sec': 3*60, # 3 min
    'obj_timeout_max_sec': 10*60, # 10 min

    # Program meta params
    #'path_to_save': PATH_RESULTS+'/GwSAA_NEW',
    #'name_to_save': 'GwSAA_NEW',
    'print_progress': True,
    #'return_results': False

}

In [ ]:
tau = 4
SKU = 35

In [ ]:
### Pre-processing
weightsmodel_name = 'rfwm_local'
samples = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples.joblib')    
weights = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights.joblib')    

In [ ]:
weights_ = {}
samples_ = {}

for t in ts:
    weights_[t] = weights[tau][t][SKU]
    samples_[t] = samples[tau][t][SKU]

In [ ]:
test = run_experiments(wsaamodel=WeightedSAA(), samples=samples_, weights=weights_, **experiment_params)    

In [ ]:
test

In [ ]:
gurobi_params = {

    'LogToConsole': 0, 
    'Threads': 1, 
    'NonConvex': 2, 
    'PSDTol': 1e-3, # 0.1%
    'MIPGap': 1e-3, # 0.1%
    'NumericFocus': 0, 
    'obj_improvement': 1e-3, # 0.1%
    'obj_timeout_sec': 3*60, # 3 min
    'obj_timeout_max_sec': 10*60, # 10 min
}

cost_params=[

    {'CR': 0.50, 'K': 100, 'u': 0.5, 'h': 1, 'b': 1},
    {'CR': 0.75, 'K': 100, 'u': 0.5, 'h': 1, 'b': 3},
    {'CR': 0.90, 'K': 100, 'u': 0.5, 'h': 1, 'b': 9}

]

In [ ]:
tau = 4
SKU = 35

In [ ]:
### Pre-processing
weightsmodel_name = 'rfwm_global'
samples = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples.joblib')    
weights = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights.joblib')    

In [ ]:
weights_ = {}
samples_ = {}

for t in ts:
    weights_[t] = weights[tau][t][SKU]
    samples_[t] = samples[tau][t][SKU]

In [ ]:
test_global = run_experiments(wsaamodel=WeightedSAA(), samples=samples_, weights=weights_, cost_params=cost_params, print_progress=True, **gurobi_params)    

In [ ]:
test_global.to_csv('test_global.csv')

In [ ]:
weights[tau][t][SKU]

In [ ]:
### Pre-processing
weightsmodel_name = 'rfwm_local'
samples = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples.joblib')    
weights = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights.joblib')    

In [ ]:
weights_ = {}
samples_ = {}

for t in ts:
    weights_[t] = weights[tau][t][SKU]
    samples_[t] = samples[tau][t][SKU]

In [ ]:
test_local = run_experiments(wsaamodel=WeightedSAA(), samples=samples_, weights=weights_, cost_params=cost_params, print_progress=True, **gurobi_params)    

In [ ]:
test_local.to_csv('test_local.csv')

In [ ]:
test_saa = run_experiments(wsaamodel=WeightedSAA(), samples=samples_, cost_params=cost_params, print_progress=True, **gurobi_params)    

In [ ]:
test_saa.to_csv('test_saa.csv')

In [ ]:
# Historical demand samples available in period t
d = samples[t]['y_train']

# Weights for Weighted SAA
if not weights is None:

    w = weights[t] 

# Weights for SAA (all weights are 1/n_samples)
else:

    w = np.array([np.repeat(1/d.shape[0],d.shape[0])]).flatten()


## Summarize weights and samples
d = d[list(w > 0)]
w = w[list(w > 0)]

df=pd.DataFrame(data=np.hstack(
    (w.reshape(w.shape[0],1), d.reshape(d.shape[0],1) if d.ndim == 1 else d)), 
        columns=[-1] + list(range(0,d.shape[1] if d.ndim > 1 else 1))).groupby(
    list(range(0,d.shape[1] if d.ndim > 1 else 1))).agg(
        w = (-1, np.sum)).reset_index()

d = np.array(df[list(range(0,d.shape[1] if d.ndim > 1 else 1))])
w = np.array([df.w]).flatten()


### Rolling Horizon Global Robust Weighted SAA (GwSAA-R)

Description ...

# Local Models

Description ...

## Data Preprocessing

We first load and pre-process the data. This includes reshaping demand time series into $(\tau+1)$-periods rolling look-ahead horizon sequences. For the local models, no demand- and feature scaling is needed.

- **ID_Data** (pd.DataFrame) stores identifiers (in particular the product (SKU) identifier and the timePeriod (sale_yearweek) identifier)
- **X_Data** (pd.DataFrame) is the 'feature matrix', i.e., each row is a feature vector $x_{j,n}$ where n is the number of training observations (rows) in the data
- **Y_Data** (pd.DataFrame) is the demand data $d_{j,n}$ (a times series per product)
- **X_Data_Columns** (pd.DataFrame) provides 'selectors' for local vs. global feature sets

In [ ]:
# Initialize preprocessing module
pp = PreProcessing()

In [ ]:
# Read data
ID_Data = pd.read_csv(PATH_DATA+'/ID_Data.csv')
X_Data = pd.read_csv(PATH_DATA+'/X_Data.csv')
X_Data_Columns = pd.read_csv(PATH_DATA+'/X_Data_Columns.csv')
Y_Data = pd.read_csv(PATH_DATA+'/Y_Data.csv')

In [ ]:
# Select features
X_Data_Columns = X_Data_Columns.loc[X_Data_Columns.Local == 'YES']
X_Data = X_Data[X_Data_Columns.Feature.values]

In [ ]:
# Ensure data is sorted by SKU and sale_yearweek for preprocessing
data = pd.concat([ID_Data, X_Data, Y_Data], axis=1).sort_values(by=['SKU', 'sale_yearweek']).reset_index(drop=True)

ID_Data = data[ID_Data.columns]
X_Data = data[X_Data.columns]
Y_Data = data[Y_Data.columns]

### Reshape to multi-period demand

We now reshape the time series of demands per product to consecutive $(\tau+1)$-periods demand vectors.

In [ ]:
# Create multi-period vectors
data = pd.concat([ID_Data, Y_Data], axis=1)
Y = {}
for tau in taus:
    Y['Y'+str(tau)] = data.groupby(['SKU']).shift(-tau)['Y']
    
Y_Data = pd.DataFrame(Y)

## Weights model

The weights model - and thus the data used, weight functions, and weights per sample - are the same for the two local models **wSAA** and **wSAA-R**. First, we tune the hyper parameters of the random forest weights model for each given look-ahead $\tau$ (as for each look-ahead $\tau$ we have a different response for the multi-output random forest regressor) and for each product (SKU) $k=1,...,M$ separately. Second, we fit all weight functions (for each look-ahead $\tau=0,...,4$ and over periods $t=1,...,T$) for each product (SKU) $k=1,...,M$ separately and generate all weights (for each look-ahead $\tau=0,...,4$, over periods $t=1,...,T$, and for each product (SKU) $k=1,...,M$ separatey).

### Tune weights model

To tune the hyper parameters of the local random forest weights model for each product (SKU) $k=1,...,M$, we use 3-fold rolling timeseries cross-validation on the training data and perform random search with 100 iterations over the specified hyper parameter search grid.

In [ ]:
# Set names
weightsmodel_cv_name = 'cv_rfwm_local'

In [ ]:
# Set parameters to tune random forest weights kernels
model_params = {
    'oob_score': True,
    'random_state': 12345,
    'n_jobs': 4,
    'verbose': 0
}

hyper_params_grid = {
    'n_estimators': [100],
    'max_depth': [None],
    'min_samples_split': [x for x in range(2, 20, 1)],  
    'min_samples_leaf': [x for x in range(2, 10, 1)],  
    'max_features': [x for x in range(8, 256, 8)],   
    'max_leaf_nodes': [None],
    'min_impurity_decrease': [0.0],
    'bootstrap': [True],
    'max_samples': [0.75, 0.80, 0.85, 0.90, 0.95, 1.00]
}    


tuning_params = {     
    'random_search': True,
    'n_iter': 100,
    'scoring': {'MSE': 'neg_mean_squared_error'},
    'return_train_score': True,
    'refit': 'MSE',
    'random_state': 12345,
    'n_jobs': 8,
    'verbose': 2
}    

In [ ]:
# Tune random forest weights models for tau=0,...,4
for tau in taus:
    
    # Initialize
    cv_results = {}
    
    # For each product (SKU)
    for SKU in SKUs:
        
        # Status
        print('........................... SKU', str(SKU), '...........................')
        
        # Select training data
        train = (ID_Data.SKU == SKU) & (ID_Data.sale_yearweek < test_start)
        rolling_horizon = [l for l in range(0,tau+1)]

        # Select training data
        ID_Data_train = ID_Data.loc[train]
        X_Data_train = X_Data.loc[train]
        Y_Data_train = Y_Data.loc[train].iloc[:,rolling_horizon]

        # Reshape to match (tau+1)-periods rolling horizon
        timePeriods = ID_Data_train.sale_yearweek
        maxTimePeriod = test_start-1

        id_train = pp.reshape_data(ID_Data_train, timePeriods, maxTimePeriod, tau)
        X_train = pp.reshape_data(X_Data_train, timePeriods, maxTimePeriod, tau)
        y_train = pp.reshape_data(Y_Data_train, timePeriods, maxTimePeriod, tau)

        # Tansfrom data to arrays
        X_train = np.array(X_train)
        y_train = np.array(y_train).flatten() if np.array(y_train).shape[1] == 1 else np.array(y_train)    

        # Initialize
        weightsmodel = RandomForestWeightsModel()

        # CV folds
        cv_folds = pp.split_timeseries_cv(n_splits=3, timePeriods=id_train.sale_yearweek)

        # CV search
        cv_results[SKU] = weightsmodel.tune(X=X_train, y=y_train, cv_folds=cv_folds, model_params=model_params, 
                                            tuning_params=tuning_params, hyper_params_grid=hyper_params_grid)

    # Save
    out = joblib.dump(cv_results, PATH_WEIGHTSMODEL+'/'+weightsmodel_cv_name+'_tau'+str(tau)+'.joblib')

### Fit weights model and generate weights

We now fit a local random forest weights model (i.e., the weight functions) for each $\tau=0,...,4$, period $t=1,...,T$, and product (SKU) $k=1,...,M$ separately (local training). Then, for each $\tau=0,...,4$, period $t=1,...,T$, and product (SKU) $k=1,...,M$ separately, we generate the weights given the test feature $x_{k,t}$. This is done *separately* for each product (SKU) $k=1,...,M$.

In [ ]:
# Set names
weightsmodel_cv_name = 'cv_rfwm_local'
weightsmodel_name = 'rfwm_local'

In [ ]:
# Initialize
samples = {}
weightfunctions = {}
weights = {}
exec_time_sec = {}
cpu_time_sec = {}
        
# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Initialize
    samples[tau] = {}
    weightfunctions[tau] = {}
    weights[tau] = {}
    exec_time_sec[tau] = {}
    cpu_time_sec[tau] = {}

    # For each period t=1,...,T
    for t in ts:
        
        # Initialize
        samples[tau][t] = {}
        weightfunctions[tau][t] = {}
        weights[tau][t] = {}
        exec_time_sec[tau][t] = {}
        cpu_time_sec[tau][t] = {}
            
        # Adjust look-ahead tau to account for end of horizon
        tau_t = min(tau,T-t)
        
        # Status
        print('#### Look-ahead tau='+str(tau)+' (adjusted to tau\'='+str(tau_t)+'), period t='+str(t)+'...')
        start_time = dt.datetime.now().replace(microsecond=0)
            
        # For each product (SKU)
        for SKU in SKUs:

            # Timer
            exec_time_sec[tau][t][SKU] = {}
            cpu_time_sec[tau][t][SKU] = {}

            # Select training and test data
            train = (ID_Data.SKU == SKU) & (ID_Data.sale_yearweek < test_start+t-1)
            test = (ID_Data.SKU == SKU) & (ID_Data.sale_yearweek == test_start+t-1)
            rolling_horizon = [l for l in range(0,tau_t+1)]

            ID_Data_train, ID_Data_test = ID_Data.loc[train], ID_Data.loc[test]
            X_Data_train, X_Data_test = X_Data.loc[train], X_Data.loc[test]
            Y_Data_train, Y_Data_test = Y_Data.loc[train].iloc[:,rolling_horizon], Y_Data.loc[test].iloc[:,rolling_horizon]

            # Reshape to match (tau+1)-periods rolling horizon
            timePeriods = ID_Data_train.sale_yearweek
            maxTimePeriod = test_start-1+t-1

            id_train = pp.reshape_data(ID_Data_train, timePeriods, maxTimePeriod, tau_t)
            X_train = pp.reshape_data(X_Data_train, timePeriods, maxTimePeriod, tau_t)
            y_train = pp.reshape_data(Y_Data_train, timePeriods, maxTimePeriod, tau_t)

            # Tansfrom data to arrays
            X_train, X_test = np.array(X_train), np.array(X_Data_test)
            y_train, y_test = np.array(y_train), np.array(Y_Data_test) 

            if tau_t == 0: y_train, y_test = y_train.flatten(), y_test.flatten() 

            # Check if fit already exists, due to the adjusted look-ahead tau_t:
            if tau_t in weightfunctions.keys():
                if t in weightfunctions[tau_t].keys():
                    if SKU in weightfunctions[tau_t][t].keys():

                        # Set weightsmodel to fitted weightsmodel already existing
                        weightsmodel = weightfunctions[tau_t][t][SKU]
                        weightfunctions[tau][t][SKU] = weightfunctions[tau_t][t][SKU]
                        exec_time_sec[tau][t][SKU]['fit'] = exec_time_sec[tau_t][t][SKU]['fit']
                        cpu_time_sec[tau][t][SKU]['fit'] = cpu_time_sec[tau_t][t][SKU]['fit']

                    else: 

                        # Timer start
                        st_exec = time.time()
                        st_cpu = time.process_time() 

                        # Initialize weights model
                        weightsmodel = RandomForestWeightsModel()

                        # Load cv results
                        weightsmodel.load_cv_result(path=PATH_WEIGHTSMODEL+'/'+weightsmodel_cv_name+'_tau'+str(tau_t)+'.joblib', SKU=SKU)

                        # Fit random forest weights model
                        weightfunctions[tau][t][SKU] = weightsmodel.fit(X=X_train, y=y_train, model_params={'n_jobs': 32, 'verbose': 0})

                        # Timer end
                        exec_time_sec[tau][t][SKU]['fit'] = time.time()-st_exec
                        cpu_time_sec[tau][t][SKU]['fit'] = time.process_time()-st_cpu

            # Check if weights already exists due to the adjusted look-ahead tau_t:
            if tau_t in weights.keys():
                if t in weights[tau_t].keys():
                    if SKU in weights[tau_t][t].keys():
                        
                        # Set weights to weights already existing
                        weights[tau][t][SKU] = weights[tau_t][t][SKU]
                        exec_time_sec[tau][t][SKU]['weights'] = exec_time_sec[tau_t][t][SKU]['weights']
                        cpu_time_sec[tau][t][SKU]['weights'] = cpu_time_sec[tau_t][t][SKU]['weights']

                    else: 

                        # Timer start
                        st_exec = time.time()
                        st_cpu = time.process_time()  

                        # Get weights
                        weights[tau][t][SKU] = weightsmodel.apply(X=X_train, x=X_test, model_params={'n_jobs': 32, 'verbose': 0}).flatten()
      
                        # Timer end
                        exec_time_sec[tau][t][SKU]['weights'] = time.time()-st_exec
                        cpu_time_sec[tau][t][SKU]['weights'] = time.process_time()-st_cpu


            # Check if samples already exists due to the adjusted look-ahead tau_t:
            if tau_t in samples.keys():
                if t in samples[tau_t].keys():
                    if SKU in samples[tau_t][t].keys():

                        # Set samples to samples already existing
                        samples[tau][t][SKU] = samples[tau_t][t][SKU]

                    else: 

                        # Store unscaled samples for each test product separately
                        samples[tau][t][SKU] = {'y_train': y_train,
                                                'y_test': y_test, 
                                                'id_train': id_train,
                                                'id_test': ID_Data_test}

        # Status
        print('...done in', dt.datetime.now().replace(microsecond=0) - start_time)    

# Save results
joblib.dump(samples, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples.joblib')    
joblib.dump(weightfunctions, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weightfunctions.joblib')    
joblib.dump(weights, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights.joblib')    
joblib.dump(exec_time_sec, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weightsmodel_exec_time_sec.joblib')  
joblib.dump(cpu_time_sec, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weightsmodel_cpu_time_sec.joblib')  

## Optimization

### Rolling Horizon Local Weighted SAA (wSAA)

Description ...

### Rolling Horizon Local Robust Weighted SAA (wSAA-R)

Description ...

# Baseline model: Rolling Horizon Local Weighted SAA (SAA)

Description ...

# Ex-post optimal model

Description ...

# ARCHIVE >>>

# Functions

## Data preparation

We first define a function that prepares the data needed for an experiment (depending on the model/approach). 

If no sampling strategy is provided via the optional argument 'sampling', no weights are retrieved, else 'global' or 'local' weights are retrieved and historical demands are prepared for 'global' or 'local' sampling, respectively. 
    
If the optional argument 'e' is provided, the function additionally outputs 'epsilon' which is the uncertainty set threshold for robust optimization.

In [ ]:
## Prepare the data to a run an experiment over the full planning horizon
def prep_data(SKU, tau, T, sale_yearweek, path_data, path_samples, **kwargs):

    """

    This function prepares the data needed for (weighted, robust) optimization. If no sampling strategy is
    provided via the optional argument 'sampling', no weights are retrieved, else 'global' or 'local' weights
    are retrieved and historical demands are prepared for 'global' or 'local' sampling, respectively. If the
    optional argument 'e' is provided, the function additionally outputs 'epsilon' which is the uncertainty
    set threshold for robust optimization.

    Arguments:

        SKU: product (SKU) identifier
        tau: length of rolling look-ahead horizon
        T: Length T of the test horizon
        sale_yearweek: Last sale_yearweek of training data
        path_data: path of data
        path_samples: path of samples

    Optional arguments: 

        sampling: Sampling strategy (either 'global', 'local'), with
            - 'global': uses weights generated with global training
            - 'local': uses weights generated with local training
        e: Robust uncertainty set threshold multiplier, with
            - int: uses e as multiplier for product's in sample standard deviation as the uncertainty set threshold 

    Output:

        y: demand data - np.array of shape (n_samples, n_periods)
        ids_train: list of selector series (True/False of length n_samples) - list with lengths of the test horizon
        ids_test: list of selector series (True/False of length n_samples) - list with lengths of the test horizon

        weights (optional): list of weights (flat np.array of length ids_train of t'th test period) - list 
        with length of test horizon
        epsilons (optional): list of epsilons - list with length of the test horizon

    """

    # Demand samples
    robj = pyreadr.read_r(path_data+'/Y_Data_mv_NEW.RData')
    y_samples = np.array(robj['Y_Data_mv'])

    # IDs of local demand samples
    robj = pyreadr.read_r(path_data+'/ID_Data_NEW.RData')
    ID_samples = robj['ID_Data']

    # IDs of local demand samples
    robj = pyreadr.read_r(path_samples+'/SKU'+str(SKU)+'/Static/TmpFiles'+
                          str(tau)+'/ID_samples_k.RDS')
    ID_samples_SKU = robj[None]

    # If sampling strategy is provided
    if 'sampling' in kwargs:

        # Weights
        with open(path_samples+'/SKU'+str(SKU)+'/Static/Weights'+
                  str(tau)+'/weights_'+kwargs['sampling']+'_ij.p', 'rb') as f:
            weighty_ij = pickle.load(f)
        del f

        # Demand samples for global sampling
        if kwargs['sampling'] == 'global':
            y = y_samples

        # Demand samples for local sampling
        if kwargs['sampling'] == 'local':
            y = y_samples[ID_samples.SKU_API == ID_samples_SKU.SKU_API[0]]

    # Default: local demand samples
    else:
        y = y_samples[ID_samples.SKU_API == ID_samples_SKU.SKU_API[0]]


    # Reshape data for each t=1...T (i.e., each period of the test horizon)
    ids_train = []
    ids_test = []

    weights = [] if 'sampling' in kwargs else None
    epsilons = [] if 'e' in kwargs else None

    # Iterate over t
    for t in range(T):

        # If sampling strategy is provided
        if 'sampling' in kwargs:

            # IDs of demand samples for global sampling
            if kwargs['sampling'] == 'global':
                ids_train = ids_train + [ID_samples.sale_yearweek < sale_yearweek+t]

            # IDs of demand samples for local sampling
            if kwargs['sampling'] == 'local':
                ids_train = ids_train + [(ID_samples.SKU_API == ID_samples_SKU.SKU_API[0]) &
                                         (ID_samples.sale_yearweek < sale_yearweek+t)]                   

            # Weights for global/local
            weights = weights + [weighty_ij[t+1]]

        # Default: IDs of demand samples for local sampling
        else:
            ids_train = ids_train + [(ID_samples.SKU_API == ID_samples_SKU.SKU_API[0]) &
                                         (ID_samples.sale_yearweek < sale_yearweek+t)]



        # IDs of demand samples for testing 
        ids_test = ids_test + [(ID_samples.SKU_API == ID_samples_SKU.SKU_API[0]) &
                                         (ID_samples.sale_yearweek == sale_yearweek+t)]


        # If e is provided, calculate robust optimization parameter epsilon
        if 'e' in kwargs:
            epsilons = epsilons + [kwargs['e']*np.std(y_samples[(ID_samples.SKU_API == ID_samples_SKU.SKU_API[0]) &
                                                                (ID_samples.sale_yearweek < sale_yearweek+t),0])]


    # Return
    return y, ids_train, ids_test, weights, epsilons

## Experiment wrapper

We now define a 'wrapper' function that iterates the experiment for a given SKU over differen cost parameter settins and lengths of the rolling look-ahead horizon tau.

If the parameter sampling is provided (either 'global' or 'local), the function uses the specified sampling strategy. Else, SAA is performed. If the multiplier 'e' for the uncertainty set threshold epsilon is provided, the function performs the robust extension.
 
Where: epsilon[t] = e *  in-sample standard deviation of the current product (SKU).

The function prepares and calls the experiment over t=1...T for each cost paramater setting and look-ahead horizon tau and then summarises the results including performance and performance meta inormation. It also saves the results in CSV format to the specified path and the function can also be used in parallel processing environments.

In [ ]:
def run_experiments(SKU, **kwargs):
    
    """
    
    Description ...
    
    
    Arguments:
    
        SKU: product (SKU) identifier
        sale_yearweek: Last sale_yearweek of training data
        T: Length T of the test horizon
        tau: List of lengths of rolling look-ahead horizons
        cost_params: dictionary/dictionary of dictionaries of cost parameters {'CR', 'K', 'u', 'h', 'b'}
        gurobi_params: dictionary of gurobi meta params {'LogToConsole', 'Threads', 'NonConvex' 
                                                         'PSDTol', 'MIPGap', 'NumericFocus',
                                                         'obj_improvement', obj_timeout_sec'}
        path: directory where results should be saved
        model_name: model name for the file to save results
        
    Optional arguments:
    
        sampling: sampling strategy (either 'global' or 'local'); performs SAA if not provided
        e: robust uncertainty set threshold multiplier; performs no robust extension if not provided
    

    """
  
    st_exec = time.time()
    st_cpu = time.process_time()
    
    # Print progress
    if kwargs['print_progress']: 
        print('SKU:', SKU)
    
    # Initialize
    rhopt = RollingHorizonOptimization()
    results = pd.DataFrame()
   
    # For each cost param setting
    for cost_params in kwargs['cost_params'].values():
        
        # Print progress
        if kwargs['print_progress']: 
            print('... cost param setting:', cost_params)
    
        # For each rolling look-ahead horizon
        for tau in kwargs['tau']:
            
            # Print progress
            if kwargs['print_progress']: 
                print('...... look-ahead horizon:', tau)
    
            ## Weighted (Robust) SAA
            if 'sampling' in kwargs:
    
                ## Weighted Robust SAA  
                if 'e' in kwargs:

                    # Prepare data
                    data = prep_data(SKU, tau, kwargs['T'], kwargs['sale_yearweek'], PATH_DATA, PATH_SAMPLES, sampling=kwargs['sampling'], e=kwargs['e'])
                    y, ids_train, ids_test, weights, epsilons = data
                    
                    # Create empty model
                    wsaamodel = RobustWeightedSAA(**kwargs['gurobi_params'])

                    # Run rolling horizon model over t=1...T
                    result = rhopt.run(y, ids_train, ids_test, tau, wsaamodel, weights=weights, epsilons=epsilons, **cost_params)

                ## Weighted SAA
                else: 
                    
                    # Prepare data
                    data = prep_data(SKU, tau, kwargs['T'], kwargs['sale_yearweek'], PATH_DATA, PATH_SAMPLES, sampling=kwargs['sampling'])
                    y, ids_train, ids_test, weights, _ = data
                    
                    # Create empty model
                    wsaamodel = WeightedSAA(**kwargs['gurobi_params'])

                    # Run rolling horizon model over t=1...T
                    result = rhopt.run(y, ids_train, ids_test, tau, wsaamodel, weights=weights, **cost_params)


            ## SAA
            else:
                
                # Prepare data
                data = prep_data(SKU, tau, kwargs['T'], kwargs['sale_yearweek'], PATH_DATA, PATH_SAMPLES)
                y, ids_train, ids_test, _, _ = data

                # Create empty model
                wsaamodel = WeightedSAA(**kwargs['gurobi_params'])

                # Run rolling horizon model over t=1...T
                result = rhopt.run(y, ids_train, ids_test, tau, wsaamodel, **cost_params)

            
            ## ToDo: ExPost
            
            # Store result
            meta = pd.DataFrame({

                'SKU': np.repeat(SKU,kwargs['T']),
                'n_periods': np.repeat(kwargs['T'],kwargs['T']),
                'tau': np.repeat(tau,kwargs['T']),
                'CR': np.repeat(cost_params['CR'],kwargs['T']),
                'LogToConsole': np.repeat(kwargs['gurobi_params']['LogToConsole'],kwargs['T']),
                'Threads': np.repeat(kwargs['gurobi_params']['Threads'],kwargs['T']),
                'NonConvex': np.repeat(kwargs['gurobi_params']['NonConvex'],kwargs['T']),
                'PSDTol': np.repeat(kwargs['gurobi_params']['PSDTol'],kwargs['T']),
                'MIPGap': np.repeat(kwargs['gurobi_params']['MIPGap'],kwargs['T']),
                'NumericFocus': np.repeat(kwargs['gurobi_params']['NumericFocus'],kwargs['T']),
                'obj_improvement': np.repeat(kwargs['gurobi_params']['obj_improvement'],kwargs['T']),
                'obj_timeout_sec': np.repeat(kwargs['gurobi_params']['obj_timeout_sec'],kwargs['T']),
                'e': np.repeat(kwargs['e'],kwargs['T']) if 'e' in kwargs else np.repeat(0,kwargs['T']),
                'epsilon': [epsilon for epsilon in epsilons] if 'e' in kwargs else np.repeat(0,kwargs['T'])
            })

            result = pd.concat([meta, result], axis=1)

            # Store
            if not results.empty:
                results = results.append(result)   
            else:
                results = pd.DataFrame(result) 

    # Save result
    save_log = results.to_csv(
        path_or_buf=kwargs['path']+'/'+kwargs['model_name']+'_SKU'+str(SKU)+(('_e'+str(kwargs['e'])) if 'e' in kwargs else '')+'.csv', 
        sep=',', index=False
    )
    
    
    # Time
    exec_time_sec = time.time() - st_exec
    cpu_time_sec = time.process_time() - st_cpu
    
    # Print progress
    if kwargs['print_progress']: 
        print('>>>> Done:',str(np.around(exec_time_sec/60,1)), 'minutes')

    
    # Returns results 
    if (kwargs['return_results'] if 'return_results' in kwargs else False):
        return results
    
    # Returns a log
    else:
        return  {'SKU': SKU, 'exec_time_sec': exec_time_sec, 'cpu_time_sec': cpu_time_sec}

**Context Manager**

This is a context manager for parellel execution with the purpose of reporting progress. 

Credits: https://stackoverflow.com/questions/24983493/tracking-progress-of-joblib-parallel-execution

In [ ]:
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

# Experiments

In [ ]:
# Set folder names as global variables
os.chdir('/home/fesc/')
global PATH_DATA, PATH_PARAMS, PATH_SAMPLES, PATH_RESULTS

PATH_DATA = '/home/fesc/MM/Data'
PATH_PARAMS  = '/home/fesc/MM/Data/Params'
PATH_SAMPLES = '/home/fesc/MM/Data/Samples'
PATH_RESULTS = '/home/fesc/MM/Data/Results'

For the models specified, the code below runs the experiment for all given products (SKUs) and over parameter settings (e.g., cost parameters, horizon parameters, etc.). In total, we have 460 products (SKUs) with each 3 different cost parameter settings varying the critical ratio (CR) of holding and backlogging cost being {CR=0.50, CR=0.75, CR=0.90) and each 5 different lengths of the rolling look-ahead horizon tau being {1,2,3,4,5}.

## (a) Rolling Horizon Global Weighted SAA

In [ ]:
# Define paramaters
params = {
            
    # Sampling strategy
    'sampling': 'global',

    # Last sale_yearweek of training data
    'sale_yearweek': 114,

    # Length T of the test horizon
    'T': 13,

    # Lengths of rolling look-ahead horizons
    'tau': [1,2,3,4,5],

    # Cost param settings
    'cost_params': {

        1: {'CR': 0.50, 'K': 100, 'u': 0.5, 'h': 1, 'b': 1},
        2: {'CR': 0.75, 'K': 100, 'u': 0.5, 'h': 1, 'b': 3},
        3: {'CR': 0.90, 'K': 100, 'u': 0.5, 'h': 1, 'b': 9}

    },

    # Gurobi meta params
    'gurobi_params': {

        'LogToConsole': 1, 
        'Threads': 1, 
        'NonConvex': 2, 
        'PSDTol': 1e-3, # 0.1%
        'MIPGap': 1e-3, # 0.1%
        'NumericFocus': 0, 
        'obj_improvement': 1e-3, # 0.1%
        'obj_timeout_sec': 3*60, # 3 min
        'obj_timeout_max_sec': 10*60, # 10 min

    },
    
    'path': PATH_RESULTS+'/GwSAA',
    'model_name': 'GwSAA',
    
    'print_progress': False,
    'return_results': False
    
}

In [ ]:
# Set path
#os.mkdir(params['path'])
       
# Specify number of cores to use for parallel execution
n_jobs = 32

# Specify range of products (SKUs) to iterate over
SKU_range = range(1,460+1)

# Run for each product (SKU) in parallel
with tqdm_joblib(tqdm(desc='Progress', total=len(SKU_range))) as progress_bar:
    resultslog = Parallel(n_jobs=n_jobs)(delayed(run_experiments)(SKU, **params)
                                         for SKU in SKU_range)

## (b) Rolling Horizon Global Robust Weighted SAA

In [ ]:
# Define paramaters
params = {
            
    # Sampling strategy
    'sampling': 'global',

    # Robust uncertainty set threshold multiplier
    'e': None,

    # Last sale_yearweek of training data
    'sale_yearweek': 114,

    # Length T of the test horizon
    'T': 13,

    # Lengths of rolling look-ahead horizons
    'tau': [1,2,3,4,5],

    # Cost param settings
    'cost_params': {

        1: {'CR': 0.50, 'K': 100, 'u': 0.5, 'h': 1, 'b': 1},
        2: {'CR': 0.75, 'K': 100, 'u': 0.5, 'h': 1, 'b': 3},
        3: {'CR': 0.90, 'K': 100, 'u': 0.5, 'h': 1, 'b': 9}

    },

    # Gurobi meta params
    'gurobi_params': {

        'LogToConsole': 1, 
        'Threads': 1, 
        'NonConvex': 2, 
        'PSDTol': 1e-3, # 0.1%
        'MIPGap': 1e-3, # 0.1%
        'NumericFocus': 3, 
        'obj_improvement': 1e-3, # 0.1%
        'obj_timeout_sec': 3*60, # 3 min
        'obj_timeout_max_sec': 10*60, # 10 min

    },
    
    'path': PATH_RESULTS+'/GwSAAR',
    'model_name': 'GwSAAR',
    
    'print_progress': False,
    'return_results': False
    
}

In [ ]:
# Set path
#os.mkdir(params['path'])

# Specify number of cores to use for parallel execution
n_jobs = 32

# Specify range of products (SKUs) to iterate over
SKU_range = range(1,460+1)

# Uncertainty set
params['e'] = 1

# Run for each product (SKU) in parallel
with tqdm_joblib(tqdm(desc='Progress', total=len(SKU_range))) as progress_bar:
    resultslog = Parallel(n_jobs=n_jobs)(delayed(run_experiments)(SKU, **params)
                                         for SKU in SKU_range)

In [ ]:
# Set path
#os.mkdir(params['path'])

# Specify number of cores to use for parallel execution
n_jobs = 32

# Specify range of products (SKUs) to iterate over
SKU_range = range(1,460+1)

# Uncertainty set
params['e'] = 3

# Run for each product (SKU) in parallel
with tqdm_joblib(tqdm(desc='Progress', total=len(SKU_range))) as progress_bar:
    resultslog = Parallel(n_jobs=n_jobs)(delayed(run_experiments)(SKU, **params)
                                         for SKU in SKU_range)

In [ ]:
# Set path
#os.mkdir(params['path'])

# Specify number of cores to use for parallel execution
n_jobs = 32

# Specify range of products (SKUs) to iterate over
SKU_range = range(1,460+1)

# Uncertainty set
params['e'] = 6

# Run for each product (SKU) in parallel
with tqdm_joblib(tqdm(desc='Progress', total=len(SKU_range))) as progress_bar:
    resultslog = Parallel(n_jobs=n_jobs)(delayed(run_experiments)(SKU, **params)
                                         for SKU in SKU_range)

In [ ]:
# Set path
#os.mkdir(params['path'])

# Specify number of cores to use for parallel execution
n_jobs = 32

# Specify range of products (SKUs) to iterate over
SKU_range = range(1,460+1)

# Uncertainty set
params['e'] = 12

# Run for each product (SKU) in parallel
with tqdm_joblib(tqdm(desc='Progress', total=len(SKU_range))) as progress_bar:
    resultslog = Parallel(n_jobs=n_jobs)(delayed(run_experiments)(SKU, **params)
                                         for SKU in SKU_range)

## (c) Rolling Horizon Local Weighted SAA

## (d) Rolling Horizon Local Robust Weighted SAA

## (e) Rolling Horizon SAA

## (f) Ex-post optimal, deterministic model

# Aggregate all results